# Pazarlama Kampanyası - Hedefli Teklifler #
Dünyadaki tüm problemler optimizasyon ile çözülebilir mi? Peki bir problemin optimizasyon ile çözülmesi için neye ihtiyaç vardır? Kısıtlı kaynak ve seçim yapabilecek alternatiflerin olması. Eğer bu ikisi varsa optimizasyon ile çözmeniz durumunda en iyiye ulaşabilirsiniz. Bu örneğimizde, müşterilerimize uygun reklam kampanyaları için kaynaklarımızı etkin kullanmaya çalışacağız. Bu örnek çözüm hazırlanırken, <a href="https://github.com/IBMDecisionOptimization/docplex-examples/blob/f540bccbaf552bca0fe4a1fb13a33aaceef72588/examples/mp/jupyter/marketing_campaign.ipynb"> Viu-Long Kong tarafından hazırlanan notebook'tan </a> faydalanılmıştır.

## Problem ##
Bu örnete hayali bir bankamız bulunmaktadır. Pazarlama departmanımız, her müşteriye doğru finansal hizmet teklifi sunup eşleştirerek, gelecekteki kampanyalarda daha karlı sonuçlar elde etmek istemektedir. Veri Bilimi departmanımız daha önce verilen tekliflere ve yanıtlara göre müşterilerin hangi finansal hizmet teklifine nasıl cevap vereceğinin olasılıklarını hesapladı. Bizim yapmamız gereken bu olasılıkları kullanarak en iyi teklif planını hesaplayacak bir model kurmamız gerekiyor. 

Bunu yaparken aşağıdaki kurallara uymamız gerekiyor.
<ul>
    <li>"Hediyeler", "bülten", "seminer" gibi araçları kullanarak yapacağız pazarlama kampanyası  için sınırlı bir bütçemiz var.</li>
    <li>Müşterilerle iletişim kurmanın en iyi yolunun hangisi olduğunu belirlemek istiyoruz.</li>
    <li>Hangi müşterilerle iletişime geçeceğimizi belirlememiz gerekiyor.</li>
</ul>
Haydi başlayalım

## DOcplex ile Çözüm ##
__Verinin Hazırlanması__<br>
Tahminler, bir müşterinin hangi teklifi kabul etme olasılığının en yüksek olduğunu ve her müşterinin ayrıntılarına bağlı olarak kabul edeceği güveni gösterir.

Temelde iki verimiz bulunmaktadır. Birinci verimiz müşterilerin ismini vermektedir. Diğer verimiz ise her bir müşterinin ürünlerimizi alma olasılıklarını berlirtmektedir.

Örneğin: (139987, "Pension", 0.13221, "Mortgage", 0.10675),  müşteri Id=139987'nin %13 oranından dolayı kesinlikle Emeklilik satın almayacağını belirtirken (140030, "Savings", 0.95678, "Pension" ", 0.84446), 140030 ID'li müşteri için oranlar %95,7 ve %84,4 olduğundan Tasarruf ve Emeklilik satın alma olasılığı daha yüksektir.

Bu örneğimizde Pandas kütüphanesini kullanacağız.

In [2]:
import pandas as pd

names = {
    139987 : "Guadalupe J. Martinez", 140030 : "Michelle M. Lopez", 140089 : "Terry L. Ridgley", 
    140097 : "Miranda B. Roush", 139068 : "Sandra J. Wynkoop", 139154 : "Roland Guérette", 139158 : "Fabien Mailhot", 
    139169 : "Christian Austerlitz", 139220 : "Steffen Meister", 139261 : "Wolfgang Sanger",
    139416 : "Lee Tsou", 139422 : "Sanaa' Hikmah Hakimi", 139532 : "Miroslav Škaroupka", 
    139549 : "George Blomqvist", 139560 : "Will Henderson", 139577 : "Yuina Ohira", 139580 : "Vlad Alekseeva", 
    139636 : "Cassio Lombardo", 139647 : "Trinity Zelaya Miramontes", 139649 : "Eldar Muravyov", 139665 : "Shu T'an", 
    139667 : "Jameel Abdul-Ghani Gerges", 139696 : "Zeeb Longoria Marrero", 139752 : "Matheus Azevedo Melo", 
    139832 : "Earl B. Wood", 139859 : "Gabrielly Sousa Martins", 139881 : "Franca Palermo"}


data = [(139987, "Pension", 0.13221, "Mortgage", 0.10675), (140030, "Savings", 0.95678, "Pension", 0.84446), (140089, "Savings", 0.95678, "Pension", 0.80233), 
                        (140097, "Pension", 0.13221, "Mortgage", 0.10675), (139068, "Pension", 0.80506, "Savings", 0.28391), (139154, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139158, "Pension", 0.13221, "Mortgage", 0.10675),(139169, "Pension", 0.13221, "Mortgage", 0.10675), (139220, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139261, "Pension", 0.13221, "Mortgage", 0.10675), (139416, "Pension", 0.13221, "Mortgage", 0.10675), (139422, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139532, "Savings", 0.95676, "Mortgage", 0.82269), (139549, "Savings", 0.16428, "Pension", 0.13221), (139560, "Savings", 0.95678, "Pension", 0.86779), 
                        (139577, "Pension", 0.13225, "Mortgage", 0.10675), (139580, "Pension", 0.13221, "Mortgage", 0.10675), (139636, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139647, "Savings", 0.28934, "Pension", 0.13221), (139649, "Pension", 0.13221, "Mortgage", 0.10675), (139665, "Savings", 0.95675, "Pension", 0.27248), 
                        (139667, "Pension", 0.13221, "Mortgage", 0.10675), (139696, "Savings", 0.16188, "Pension", 0.13221), (139752, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139832, "Savings", 0.95678, "Pension", 0.83426), (139859, "Savings", 0.95678, "Pension", 0.75925), (139881, "Pension", 0.13221, "Mortgage", 0.10675)]

products = ["Car loan", "Savings", "Mortgage", "Pension"]
productValue = [100, 200, 300, 400]
budgetShare = [0.6, 0.1, 0.2, 0.1]

availableBudget = 500
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), ("newsletter", 15.0, 0.05), ("seminar", 23.0, 0.30)], columns=["name", "cost", "factor"])

Pandas kullanarak, offers veri yapısını oluşturuyoruz ve bu veri yapısına müşteri isimlerini getiriyoruz

In [8]:
offers = pd.DataFrame(data=data, index=range(0, len(data)), columns=["customerid", "Product1", "Confidence1", "Product2", "Confidence2"])
offers.insert(0,'name',pd.Series(names[i[0]] for i in data))

In [10]:
offers.head(5)

,name,customerid,Product1,Confidence1,Product2,Confidence2
0,Guadalupe J. Martinez,139987,Pension,0.13221,Mortgage,0.10675
1,Michelle M. Lopez,140030,Savings,0.95678,Pension,0.84446
2,Terry L. Ridgley,140089,Savings,0.95678,Pension,0.80233
3,Miranda B. Roush,140097,Pension,0.13221,Mortgage,0.10675
4,Sandra J. Wynkoop,139068,Pension,0.80506,Savings,0.28391


__Modelin kurulması__

In [12]:
import docplex.mp
import cplex
from docplex.mp.model import Model

In [13]:
mdl = Model(name="marketing_campaign")

__Karar Değişkenlerinin Tanımlanması__

__ChannelVars:__ _tamsayılı karar değişkeni (binary)_ - Bir müşteriye belirli bir kanal aracılığıyla belirli bir ürün için teklif yapılıp yapılmayacağını temsil eder. <br>
__Totaloffers:__ _tamsayı karar değişkeni_ - Yapılan tekliflerin toplam sayısını temsil eder.<br>
__budgetSpent:__ _sürekli değişken_ - Yapılan tekliflerin toplam maliyetini temsil eder.<br>

In [14]:
offersR = range(0, len(offers))
productsR = range(0, len(products))
channelsR = range(0, len(channels))

channelVars = mdl.binary_var_cube(offersR, productsR, channelsR)
totaloffers = mdl.integer_var(lb=0)
budgetSpent = mdl.continuous_var()

__Kısıtların Tanımlanması__

- Her bir müşteri için sdaece bir ürün sunulabilir.
- Bütçe hesaplanmalı ve bir sınır koyulmalıdır.
- Yapılan teklif sayısı hesaplanmalı ve diğer hesaplar içinde kullanıma hazır hale getirilmelidir.

In [15]:
# Only 1 product is offered to each customer     
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in productsR for c in channelsR) <=1
                   for o in offersR)

mdl.add_constraint( totaloffers == mdl.sum(channelVars[o,p,c] 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR) )

mdl.add_constraint( budgetSpent == mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"] 
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR) )

# Balance the offers among products   
for p in productsR:
    mdl.add_constraint( mdl.sum(channelVars[o,p,c] for o in offersR for c in channelsR) 
                       <= budgetShare[p] * totaloffers )
            
# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"] 
                            for o in offersR 
                            for p in productsR 
                            for c in channelsR)  <= availableBudget )  

mdl.print_information()

Model: marketing_campaign
 - number of variables: 326
   - binary=324, integer=1, continuous=1
 - number of constraints: 34
   - linear=34
 - parameters: defaults
 - objective: none
 - problem type is: MILP


__Amaç Fonksiyonunun Belirlenmesi__

Geliri maksimize edecek amaç fonksiyonunu yazacağız. Her bir müşteri için 2 ürün ihtimali olduğu için, amaç fonksiyonumuzu da buna göre yazıyoruz. Binary değişkenimiz sadece bir tanesini seçtiği için, bir üründe 1 diğer üründe 0 olacaktır. Sonuç olarak, tüm değişkenler ve veriler kullanarak geliri maksimize eden fonksiyonumuzu yazacağız.

In [16]:
mdl.maximize(
    mdl.sum( channelVars[idx,p,idx2] * c.factor * productValue[p]* o.Confidence1  
            for p in productsR 
            for idx,o in offers[offers['Product1'] == products[p]].iterrows()  
            for idx2, c in channels.iterrows())
    +
    mdl.sum( channelVars[idx,p,idx2] * c.factor * productValue[p]* o.Confidence2 
            for p in productsR 
            for idx,o in offers[offers['Product2'] == products[p]].iterrows() 
            for idx2, c in channels.iterrows())
    )

__Modelin Çözümü__

In [24]:
s = mdl.solve()
assert s, "No Solution !!!"

__Sonuçların Analiz Edilmesi__

İlk olarak, Müşteri Başına Optimal Pazarlama Kanalını görüntüleyelim.

In [25]:
report = [(channels.at[c, "name"], products[p], names[offers.at[o, "customerid"]]) 
          for c in channelsR 
          for p in productsR 
          for o in offersR  if channelVars[o,p,c].solution_value==1]


In [26]:
print("Marketing plan has {0} offers costing {1}".format(totaloffers.solution_value, budgetSpent.solution_value))

report_bd = pd.DataFrame(report, columns=['channel', 'product', 'customer'])
display(report_bd)

Marketing plan has 19.999999999999982 offers costing 364.0


,channel,product,customer
0,newsletter,Car loan,Fabien Mailhot
1,newsletter,Car loan,Christian Austerlitz
2,newsletter,Car loan,Lee Tsou
3,newsletter,Car loan,Sanaa' Hikmah Hakimi
4,newsletter,Car loan,George Blomqvist
5,newsletter,Car loan,Yuina Ohira
6,newsletter,Car loan,Vlad Alekseeva
7,newsletter,Car loan,Cassio Lombardo
8,newsletter,Car loan,Trinity Zelaya Miramontes
9,newsletter,Car loan,Eldar Muravyov


Şimdi de Seminer'e bakalım.

In [27]:
display(report_bd[report_bd['channel'] == "seminar"].drop('channel',1))

,product,customer
12,Savings,Terry L. Ridgley
13,Savings,Gabrielly Sousa Martins
14,Mortgage,Miranda B. Roush
15,Mortgage,Miroslav Škaroupka
16,Mortgage,Matheus Azevedo Melo
17,Mortgage,Franca Palermo
18,Pension,Michelle M. Lopez
19,Pension,Will Henderson


## Sonuç ##
Bu eğitimde, binary karar değişkeni kullanarak bir modelleme örneği çözdük. Pandas kütüphanesinin modellemede kullanılması ile ilgili de güzel bir örnek oldu. Bir sonraki eğitimde görüşmek üzere.

__Sabri Suyunu__